# Лабораторная работа №3  
## Проведение исследований с решающим деревом (Decision Tree)

В лабораторной работе исследуются модели **решающего дерева**:
- **DecisionTreeClassifier** для задачи классификации (loan_status);
- **DecisionTreeRegressor** для задачи регрессии (song_popularity).


## Импорт библиотек

In [28]:
import os
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, KFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    root_mean_squared_error, mean_absolute_error, r2_score
)

RANDOM_STATE = 42

## 1. Загрузка данных и формирование выборок

- отделяется целевая переменная;
- исключаются служебные идентификаторы (`customer_id`, `song_name`);
- определяется список категориальных и числовых признаков;
- данные делятся на train/test.

In [24]:
# Пути к данным
LOAN_PATH = os.path.join("data", "Loan_approval_data_2025.csv")
SONG_PATH = os.path.join("data", "song_data.csv")

# Загрузка датасетов
loan_df = pd.read_csv(LOAN_PATH)
song_df = pd.read_csv(SONG_PATH)

# --- Classification ---
y_cls = loan_df["loan_status"].astype(int)
X_cls = loan_df.drop(columns=["loan_status"])

if "customer_id" in X_cls.columns:
    X_cls = X_cls.drop(columns=["customer_id"])

cat_cols_cls = X_cls.select_dtypes(include=["object"]).columns.tolist()
num_cols_cls = X_cls.select_dtypes(exclude=["object"]).columns.tolist()

Xc_train, Xc_test, yc_train, yc_test = train_test_split(
    X_cls, y_cls, test_size=0.2, random_state=RANDOM_STATE, stratify=y_cls
)

# --- Regression ---
y_reg = song_df["song_popularity"].astype(float)
X_reg = song_df.drop(columns=["song_popularity"])

if "song_name" in X_reg.columns:
    X_reg = X_reg.drop(columns=["song_name"])

cat_cols_reg = X_reg.select_dtypes(include=["object"]).columns.tolist()
num_cols_reg = X_reg.select_dtypes(exclude=["object"]).columns.tolist()

Xr_train, Xr_test, yr_train, yr_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=RANDOM_STATE
)

print("Classification:", Xc_train.shape, Xc_test.shape, "| target mean:", yc_train.mean())
print("Regression:", Xr_train.shape, Xr_test.shape, "| target mean:", yr_train.mean())

Classification: (40000, 18) (10000, 18) | target mean: 0.55045
Regression: (15068, 13) (3767, 13) | target mean: 53.06099017786037


## 2. Бейзлайн и оценка качества (Decision Tree)

Для деревьев масштабирование признаков не является обязательным (модель основана на разбиениях),
но для корректной работы с категориальными переменными выполняется One-Hot Encoding.

Бейзлайн:
- DecisionTreeClassifier с параметрами по умолчанию (фиксируется random_state);
- DecisionTreeRegressor с параметрами по умолчанию (фиксируется random_state).

Затем рассчитываются метрики:
- **классификация**: Accuracy, Precision, Recall, F1, ROC-AUC;
- **регрессия**: RMSE, MAE, R².

In [25]:
# Preprocessing: impute + one-hot for categorical; impute for numeric
num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median"))
])

cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

prep_cls = ColumnTransformer([
    ("num", num_pipe, num_cols_cls),
    ("cat", cat_pipe, cat_cols_cls)
])

prep_reg = ColumnTransformer([
    ("num", num_pipe, num_cols_reg),
    ("cat", cat_pipe, cat_cols_reg)
])

baseline_cls = Pipeline([
    ("prep", prep_cls),
    ("model", DecisionTreeClassifier(random_state=RANDOM_STATE))
])

baseline_reg = Pipeline([
    ("prep", prep_reg),
    ("model", DecisionTreeRegressor(random_state=RANDOM_STATE))
])

baseline_cls.fit(Xc_train, yc_train)
baseline_reg.fit(Xr_train, yr_train)

yc_pred = baseline_cls.predict(Xc_test)
yc_proba = baseline_cls.predict_proba(Xc_test)[:, 1]

yr_pred = baseline_reg.predict(Xr_test)

baseline_cls_res = {
    "accuracy": accuracy_score(yc_test, yc_pred),
    "precision": precision_score(yc_test, yc_pred, zero_division=0),
    "recall": recall_score(yc_test, yc_pred, zero_division=0),
    "f1": f1_score(yc_test, yc_pred, zero_division=0),
    "roc_auc": roc_auc_score(yc_test, yc_proba)
}

baseline_reg_res = {
    "rmse": root_mean_squared_error(yr_test, yr_pred),
    "mae": mean_absolute_error(yr_test, yr_pred),
    "r2": r2_score(yr_test, yr_pred)
}

print("Baseline classification:", baseline_cls_res)
print("Baseline regression:", baseline_reg_res)

Baseline classification: {'accuracy': 0.8716, 'precision': 0.8808879263670818, 'recall': 0.8866485013623978, 'f1': 0.8837588267246062, 'roc_auc': np.float64(0.8699093452306984)}
Baseline regression: {'rmse': 23.825798540585282, 'mae': 15.846939904180413, 'r2': -0.17754845760785476}


## 3. Улучшение бейзлайна: гипотезы и проверка

### Гипотезы для классификации
1) Ограничение глубины дерева и минимального числа объектов в листе снижает переобучение → улучшает ROC-AUC/F1.  
2) Использование отсечения по стоимости сложности (`ccp_alpha`) может дополнительно уменьшить переобучение.

### Гипотезы для регрессии
1) Аналогично, ограничение глубины/минимального размера листа уменьшает дисперсию → снижает RMSE.  
2) Параметр `max_features` может улучшать обобщение за счёт уменьшения корреляции разбиений.

Ниже проводится GridSearchCV:
- для классификации оптимизация по ROC-AUC;
- для регрессии оптимизация по neg-RMSE.

In [26]:
# --- Tuning: Classification ---
param_grid_cls = {
    "model__criterion": ["gini", "entropy", "log_loss"],
    "model__max_depth": [None, 5, 10],
    "model__min_samples_split": [2, 5, 10, 20],
    "model__min_samples_leaf": [1, 2, 5, 10],
    "model__ccp_alpha": [0.0, 1e-3]
}


tuned_cls = Pipeline([
    ("prep", prep_cls),
    ("model", DecisionTreeClassifier(random_state=RANDOM_STATE))
])

cv_cls = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

gs_cls = GridSearchCV(
    tuned_cls,
    param_grid=param_grid_cls,
    cv=cv_cls,
    scoring="roc_auc",
    n_jobs=-1
)
gs_cls.fit(Xc_train, yc_train)

best_cls = gs_cls.best_estimator_
print("Best cls params:", gs_cls.best_params_)
print("Best CV ROC-AUC:", gs_cls.best_score_)

yc_pred2 = best_cls.predict(Xc_test)
yc_proba2 = best_cls.predict_proba(Xc_test)[:, 1]

improved_cls_res = {
    "accuracy": accuracy_score(yc_test, yc_pred2),
    "precision": precision_score(yc_test, yc_pred2, zero_division=0),
    "recall": recall_score(yc_test, yc_pred2, zero_division=0),
    "f1": f1_score(yc_test, yc_pred2, zero_division=0),
    "roc_auc": roc_auc_score(yc_test, yc_proba2)
}
print("Improved classification:", improved_cls_res)

# --- Tuning: Regression ---
param_grid_reg = {
    "model__criterion": ["squared_error", "friedman_mse"],
    "model__max_depth": [3, 5, 8, 12],
    "model__min_samples_split": [2, 5, 10, 20],
    "model__min_samples_leaf": [1, 2, 5, 10],
    "model__max_features": [None, "sqrt", "log2"],
    "model__ccp_alpha": [0.0, 1e-3]
}

tuned_reg = Pipeline([
    ("prep", prep_reg),
    ("model", DecisionTreeRegressor(random_state=RANDOM_STATE))
])

cv_reg = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

gs_reg = GridSearchCV(
    tuned_reg,
    param_grid=param_grid_reg,
    cv=cv_reg,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1
)
gs_reg.fit(Xr_train, yr_train)

best_reg = gs_reg.best_estimator_
print("Best reg params:", gs_reg.best_params_)
print("Best CV -RMSE:", gs_reg.best_score_)

yr_pred2 = best_reg.predict(Xr_test)

improved_reg_res = {
    "rmse": root_mean_squared_error(yr_test, yr_pred2),
    "mae": mean_absolute_error(yr_test, yr_pred2),
    "r2": r2_score(yr_test, yr_pred2)
}
print("Improved regression:", improved_reg_res)

Best cls params: {'model__ccp_alpha': 0.0, 'model__criterion': 'entropy', 'model__max_depth': 10, 'model__min_samples_leaf': 10, 'model__min_samples_split': 2}
Best CV ROC-AUC: 0.9513020143940724
Improved classification: {'accuracy': 0.888, 'precision': 0.8896392393815532, 'recall': 0.9093551316984559, 'f1': 0.8993891484010061, 'roc_auc': np.float64(0.9532776654654128)}
Best reg params: {'model__ccp_alpha': 0.0, 'model__criterion': 'squared_error', 'model__max_depth': 8, 'model__max_features': None, 'model__min_samples_leaf': 5, 'model__min_samples_split': 20}
Best CV -RMSE: -20.994976298583495
Improved regression: {'rmse': 20.946312882363923, 'mae': 16.475162067214562, 'r2': 0.08987922013665017}


## 4. Имплементация решающего дерева

Далее реализуются версии решающего дерева:
- для классификации: критерий Джини (Gini);
- для регрессии: MSE.

Из-за вычислительной сложности самописное дерево обучается на подвыборке. Для сравнения используются те же метрики, что и для sklearn.


## 5. Подготовка данных

In [29]:

# чтобы результаты были воспроизводимыми
rng = np.random.default_rng(RANDOM_STATE)

def to_dense(X):
    return X.toarray() if sparse.issparse(X) else np.asarray(X)

def sample_indices(n, max_n=6000, random_state=42):
    # подвыборка для ускорения самописного дерева
    if n <= max_n:
        return np.arange(n)
    rng_local = np.random.default_rng(random_state)
    return rng_local.choice(n, size=max_n, replace=False)

# --- Классификация: подготовка матриц ---
Xc_tr = prep_cls.fit_transform(Xc_train)
Xc_te = prep_cls.transform(Xc_test)

yc_tr = np.asarray(yc_train).astype(int)
yc_te = np.asarray(yc_test).astype(int)

Xc_tr_arr = to_dense(Xc_tr)
Xc_te_arr = to_dense(Xc_te)

# --- Регрессия: подготовка матриц ---
Xr_tr = prep_reg.fit_transform(Xr_train)
Xr_te = prep_reg.transform(Xr_test)

yr_tr = np.asarray(yr_train).astype(float)
yr_te = np.asarray(yr_test).astype(float)

Xr_tr_arr = to_dense(Xr_tr)
Xr_te_arr = to_dense(Xr_te)

print("CLS:", Xc_tr_arr.shape, Xc_te_arr.shape)
print("REG:", Xr_tr_arr.shape, Xr_te_arr.shape)



CLS: (40000, 27) (10000, 27)
REG: (15068, 13) (3767, 13)


## 6. Имплементация общей базы

In [30]:
class _Node:
    __slots__ = ("feature", "threshold", "left", "right", "value")
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value  # для листа: класс (int) или среднее (float)

class MyDecisionTreeBase:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1, max_features=None, random_state=42):
        self.max_depth = max_depth
        self.min_samples_split = int(min_samples_split)
        self.min_samples_leaf = int(min_samples_leaf)
        self.max_features = max_features
        self.random_state = int(random_state)
        self.root_ = None
        self.n_features_ = None
        self.rng_ = np.random.default_rng(self.random_state)

    def _feature_subset(self, n_features):
        if self.max_features is None:
            return np.arange(n_features)
        if self.max_features == "sqrt":
            k = max(1, int(np.sqrt(n_features)))
        elif self.max_features == "log2":
            k = max(1, int(np.log2(n_features)))
        elif isinstance(self.max_features, int):
            k = max(1, min(n_features, self.max_features))
        else:
            k = n_features
        return self.rng_.choice(n_features, size=k, replace=False)

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y)
        self.n_features_ = X.shape[1]
        self.root_ = self._build(X, y, depth=0)
        return self

    def _build(self, X, y, depth):
        # стоп-условия
        n = X.shape[0]
        if n < self.min_samples_split:
            return _Node(value=self._leaf_value(y))
        if self.max_depth is not None and depth >= self.max_depth:
            return _Node(value=self._leaf_value(y))

        # если узел "чистый" (в классификации) или нет смысла делить (в регрессии)
        if self._is_pure(y):
            return _Node(value=self._leaf_value(y))

        feat_ids = self._feature_subset(self.n_features_)
        best = self._best_split(X, y, feat_ids)

        if best is None:
            return _Node(value=self._leaf_value(y))

        f, thr = best
        mask = X[:, f] <= thr
        n_left = int(mask.sum())
        n_right = n - n_left

        if n_left < self.min_samples_leaf or n_right < self.min_samples_leaf:
            return _Node(value=self._leaf_value(y))

        left = self._build(X[mask], y[mask], depth + 1)
        right = self._build(X[~mask], y[~mask], depth + 1)
        return _Node(feature=f, threshold=thr, left=left, right=right, value=None)

    def predict(self, X):
        X = np.asarray(X, dtype=float)
        out = np.empty(X.shape[0], dtype=float)
        for i in range(X.shape[0]):
            out[i] = self._predict_one(X[i], self.root_)
        return out

    def _predict_one(self, x, node):
        while node.value is None:
            if x[node.feature] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node.value

    # --- методы, которые переопределяются в наследниках ---
    def _leaf_value(self, y):
        raise NotImplementedError

    def _is_pure(self, y):
        raise NotImplementedError

    def _best_split(self, X, y, feat_ids):
        raise NotImplementedError


## 7. дерево классификации (Gini)

In [31]:
class MyDecisionTreeClassifier(MyDecisionTreeBase):
    def _leaf_value(self, y):
        # мажоритарный класс
        vals, cnts = np.unique(y, return_counts=True)
        return int(vals[np.argmax(cnts)])

    def _is_pure(self, y):
        return np.unique(y).size == 1

    def _gini(self, y):
        _, cnts = np.unique(y, return_counts=True)
        p = cnts / cnts.sum()
        return 1.0 - np.sum(p * p)

    def _best_split(self, X, y, feat_ids):
        n = X.shape[0]
        base = self._gini(y)
        best_gain = 0.0
        best = None

        for f in feat_ids:
            x = X[:, f]
            # кандидаты порогов (берём уникальные значения, но ограничим)
            uniq = np.unique(x)
            if uniq.size <= 1:
                continue
            # чтобы не было слишком долго: максимум 50 порогов
            if uniq.size > 50:
                qs = np.linspace(0.05, 0.95, 50)
                thr_list = np.quantile(uniq, qs)
            else:
                thr_list = (uniq[:-1] + uniq[1:]) / 2.0

            for thr in thr_list:
                mask = x <= thr
                if mask.sum() < self.min_samples_leaf or (~mask).sum() < self.min_samples_leaf:
                    continue
                g_left = self._gini(y[mask])
                g_right = self._gini(y[~mask])
                w = mask.mean()
                impur = w * g_left + (1 - w) * g_right
                gain = base - impur
                if gain > best_gain:
                    best_gain = gain
                    best = (f, float(thr))

        return best

    def predict_proba(self, X):
        # простая вероятность по голосованию листа (0/1)
        pred = self.predict(X).astype(int)
        proba1 = pred.astype(float)
        proba0 = 1.0 - proba1
        return np.vstack([proba0, proba1]).T

print("MyDecisionTreeClassifier defined.")


MyDecisionTreeClassifier defined.


## 8. дерево регрессии

In [32]:
class MyDecisionTreeRegressor(MyDecisionTreeBase):
    def _leaf_value(self, y):
        return float(np.mean(y))

    def _is_pure(self, y):
        # для регрессии считаем "чистым", если дисперсия почти нулевая
        return float(np.var(y)) < 1e-12

    def _mse(self, y):
        if y.size == 0:
            return 0.0
        m = float(np.mean(y))
        return float(np.mean((y - m) ** 2))

    def _best_split(self, X, y, feat_ids):
        base = self._mse(y)
        best_gain = 0.0
        best = None

        for f in feat_ids:
            x = X[:, f]
            uniq = np.unique(x)
            if uniq.size <= 1:
                continue
            # ограничим пороги
            if uniq.size > 50:
                qs = np.linspace(0.05, 0.95, 50)
                thr_list = np.quantile(uniq, qs)
            else:
                thr_list = (uniq[:-1] + uniq[1:]) / 2.0

            for thr in thr_list:
                mask = x <= thr
                if mask.sum() < self.min_samples_leaf or (~mask).sum() < self.min_samples_leaf:
                    continue
                mse_left = self._mse(y[mask])
                mse_right = self._mse(y[~mask])
                w = mask.mean()
                mse_split = w * mse_left + (1 - w) * mse_right
                gain = base - mse_split
                if gain > best_gain:
                    best_gain = gain
                    best = (f, float(thr))

        return best

print("MyDecisionTreeRegressor defined.")


MyDecisionTreeRegressor defined.


## 9. Обучение/оценка своих моделей

In [33]:

# --- подвыборка для ускорения ---
idx_c = sample_indices(Xc_tr_arr.shape[0], max_n=6000, random_state=RANDOM_STATE)
idx_r = sample_indices(Xr_tr_arr.shape[0], max_n=6000, random_state=RANDOM_STATE)

# --- MyTree baseline params (похоже на бейзлайн sklearn) ---
my_tree_cls_base = MyDecisionTreeClassifier(
    max_depth=None, min_samples_split=2, min_samples_leaf=1, max_features=None, random_state=RANDOM_STATE
).fit(Xc_tr_arr[idx_c], yc_tr[idx_c])

yc_pred_m = my_tree_cls_base.predict(Xc_te_arr).astype(int)
yc_proba_m = my_tree_cls_base.predict_proba(Xc_te_arr)[:, 1]

my_baseline_cls_res = {
    "accuracy": accuracy_score(yc_te, yc_pred_m),
    "precision": precision_score(yc_te, yc_pred_m, zero_division=0),
    "recall": recall_score(yc_te, yc_pred_m, zero_division=0),
    "f1": f1_score(yc_te, yc_pred_m, zero_division=0),
    "roc_auc": roc_auc_score(yc_te, yc_proba_m),
}
print("MyTree baseline (classification):", my_baseline_cls_res)

my_tree_reg_base = MyDecisionTreeRegressor(
    max_depth=None, min_samples_split=2, min_samples_leaf=1, max_features=None, random_state=RANDOM_STATE
).fit(Xr_tr_arr[idx_r], yr_tr[idx_r])

yr_pred_m = my_tree_reg_base.predict(Xr_te_arr)

my_baseline_reg_res = {
    "rmse": root_mean_squared_error(yr_te, yr_pred_m),
    "mae": mean_absolute_error(yr_te, yr_pred_m),
    "r2": r2_score(yr_te, yr_pred_m),
}
print("MyTree baseline (regression):   ", my_baseline_reg_res)


MyTree baseline (classification): {'accuracy': 0.833, 'precision': 0.8469332368373439, 'recall': 0.8503178928247048, 'f1': 0.8486221899927484, 'roc_auc': np.float64(0.8310543857894381)}
MyTree baseline (regression):    {'rmse': 26.780227019996392, 'mae': 19.21166268471817, 'r2': -0.4876897333309129}


## 10. улучшенный самописный вариант 

Здесь логика простая: берём техники улучшенного бейзлайна как ограничение глубины + увеличение min_samples_leaf + выбор max_features, чтобы уменьшить переобучение.

In [34]:
# Улучшенные параметры (идея: меньше переобучения)
my_tree_cls_imp = MyDecisionTreeClassifier(
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=10,
    max_features="sqrt",
    random_state=RANDOM_STATE
).fit(Xc_tr_arr[idx_c], yc_tr[idx_c])

yc_pred_m2 = my_tree_cls_imp.predict(Xc_te_arr).astype(int)
yc_proba_m2 = my_tree_cls_imp.predict_proba(Xc_te_arr)[:, 1]

my_improved_cls_res = {
    "accuracy": accuracy_score(yc_te, yc_pred_m2),
    "precision": precision_score(yc_te, yc_pred_m2, zero_division=0),
    "recall": recall_score(yc_te, yc_pred_m2, zero_division=0),
    "f1": f1_score(yc_te, yc_pred_m2, zero_division=0),
    "roc_auc": roc_auc_score(yc_te, yc_proba_m2),
}
print("MyTree improved (classification):", my_improved_cls_res)

my_tree_reg_imp = MyDecisionTreeRegressor(
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=10,
    max_features="sqrt",
    random_state=RANDOM_STATE
).fit(Xr_tr_arr[idx_r], yr_tr[idx_r])

yr_pred_m2 = my_tree_reg_imp.predict(Xr_te_arr)

my_improved_reg_res = {
    "rmse": root_mean_squared_error(yr_te, yr_pred_m2),
    "mae": mean_absolute_error(yr_te, yr_pred_m2),
    "r2": r2_score(yr_te, yr_pred_m2),
}
print("MyTree improved (regression):   ", my_improved_reg_res)


MyTree improved (classification): {'accuracy': 0.8135, 'precision': 0.8408239700374532, 'recall': 0.815622161671208, 'f1': 0.8280313508529276, 'roc_auc': np.float64(0.8132615813917775)}
MyTree improved (regression):    {'rmse': 21.940023724020975, 'mae': 17.374964420720254, 'r2': 0.0014770685134980477}


## 11. Сравнение результатов и выводы

In [35]:
import pandas as pd

cls_compare = pd.DataFrame(
    [baseline_cls_res, improved_cls_res, my_baseline_cls_res, my_improved_cls_res],
    index=["sk_baseline", "sk_improved", "my_baseline", "my_improved"]
)

reg_compare = pd.DataFrame(
    [baseline_reg_res, improved_reg_res, my_baseline_reg_res, my_improved_reg_res],
    index=["sk_baseline", "sk_improved", "my_baseline", "my_improved"]
)

display(cls_compare)
display(reg_compare)


,accuracy,precision,recall,f1,roc_auc
sk_baseline,0.8716,0.880888,0.886649,0.883759,0.869909
sk_improved,0.8880,0.889639,0.909355,0.899389,0.953278
my_baseline,0.8330,0.846933,0.850318,0.848622,0.831054
my_improved,0.8135,0.840824,0.815622,0.828031,0.813262


,rmse,mae,r2
sk_baseline,23.825799,15.846940,-0.177548
sk_improved,20.946313,16.475162,0.089879
my_baseline,26.780227,19.211663,-0.487690
my_improved,21.940024,17.374964,0.001477


## Выводы по лабораторной работе №3 (Решающее дерево)

В данной лабораторной работе были исследованы модели решающего дерева для задач классификации и регрессии, включая библиотечные реализации `sklearn` и самописные модели.

### Классификация (loan_status)

Подбор гиперпараметров решающего дерева позволил улучшить качество классификации.  
Улучшенная модель `sk_improved` показала рост всех основных метрик по сравнению с бейзлайном:
- accuracy увеличилась с **0.872 → 0.888**;
- F1-score вырос с **0.884 → 0.899**;
- ROC-AUC повысился с **0.870 → 0.953**, что говорит о лучшей способности модели различать классы.

Самописная реализация решающего дерева работает корректно, однако заметно уступает библиотечной версии:
- значения accuracy и ROC-AUC ниже, чем у моделей `sklearn`;
- улучшение самописной модели дало незначительный эффект.

Это объясняется отсутствием оптимизаций и упрощённой логикой построения дерева.

### Регрессия (song_popularity)

Для задачи регрессии решающие деревья показали более слабые результаты.  
Даже после подбора гиперпараметров модель `sk_improved` демонстрирует:
- снижение RMSE (с **23.83 → 20.95**);
- рост R² до **0.09**, однако объясняемая дисперсия остаётся низкой.

Самописная регрессионная модель показала худшие результаты:
- отрицательные значения R² указывают на слабую способность модели описывать зависимость;
- высокая чувствительность дерева к шуму приводит к переобучению.

### Итог

Решающие деревья хорошо подходят для задач классификации и выигрывают от настройки гиперпараметров.  
В задаче регрессии их эффективность ограничена, особенно без ансамблевых методов.  
Библиотечные реализации `sklearn` значительно превосходят самописные модели за счёт оптимизаций и устойчивых алгоритмов построения деревьев.
